In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv("english.csv")

In [3]:
# rename columns to lowercase
columns = dataset.columns
columns = [col.lower() for col in columns]
dataset.columns = columns

In [4]:
intents = dataset[["intent"]]

In [5]:
intents = intents.dropna()

In [6]:
list_intents = intents["intent"].values.tolist()

In [7]:
length = len(dataset)

In [8]:
# fill missing values in intent column
k = -1
for i in range(length):
    if pd.isnull(dataset.loc[i,"intent"]):
        dataset.loc[i,"intent"] = list_intents[k]
    else:
        k+=1

In [9]:
entities = dataset[["entity"]]

In [10]:
entities = entities.dropna()

In [11]:
list_entities = entities["entity"].values.tolist()

In [12]:
# fill missing values in entity column
entity_index = -1
for i in range(length):
    if pd.isnull(dataset.loc[i,"entity"]):
        dataset.loc[i,"entity"] = list_entities[entity_index]
    else:
        entity_index += 1

In [13]:
dataset_intent_entity = dataset[["intent","entity"]]

In [14]:
intent_entity = dataset_intent_entity.groupby(["intent","entity"]).size()

In [15]:
intent_entity_df = intent_entity.reset_index()

In [16]:
# create dictionary of intent and entity
dict_intent_entity = {}
length_ie = len(intent_entity_df)
for i in range(length_ie):
    intent = intent_entity_df.loc[i,"intent"].lower()
    entity = intent_entity_df.loc[i,"entity"]
    flist_entity=entity.lower().split(",")
    flist_entity = [x.strip() for x in flist_entity]
    list_answer_en = dataset[dataset["entity"]==entity][["eng answer"]].dropna()["eng answer"].values.tolist()
    list_answer_km = dataset[dataset["entity"]==entity][["kh answer"]].dropna()["kh answer"].values.tolist()
    if intent not in dict_intent_entity:
        dict_intent_entity[intent] = {}
    dict_intent_entity[intent][tuple(flist_entity)] = {"en":list_answer_en,"km":list_answer_km}

In [17]:
dict_intent_entity

{'askforadmission': {('cost',
   'charge',
   'តម្លៃ',
   'ថ្លៃ'): {'en': ['School will charge after students submit register form.'], 'km': ['សាលានឹងគិតថ្លៃបន្ទាប់ពីសិស្សដាក់ទម្រង់ចុះឈ្មោះ។']},
  ('document',
   'ឯកសារ'): {'en': ['The document that school need is _student photo 4x6 _nation ID card _Certificate grade 12. For more infomation https://www.cadt.edu.kh/academic/admission/'], 'km': ['ឯកសារដែលសាលាត្រូវការគឺ _រូបថតសិស្ស 4x6 _អត្តសញ្ញាណប័ណ្ណសញ្ជាតិខ្មែរ _វិញ្ញាបនបត្រថ្នាក់ទី១២។ នេះជាតំណភ្ជាប់ដែលអ្នកអាចចូលទៅកាន់ោ: https://www.cadt.edu.kh/academic/admission/']},
  ('register',
   'online',
   'offline',
   'physical',
   'អ៊ីនធឺណិត',
   'ដោយផ្ទាល់'): {'en': ['For your convenience, we offer online and in-person registration options.'], 'km': ['ដើម្បីភាពងាយស្រួលរបស់អ្នក យើងផ្តល់ជូននូវជម្រើសចុះឈ្មោះតាមអ៊ីនធឺណិត និងដោយផ្ទាល់។']},
  ('ឯណា',
   'ដូចម្តេច',
   'how to',
   'where'): {'en': ['Here is the link you can visit:  https://www.idt.edu.kh/admission-2/'], 'km': ['នេះជាតំណភ្ជាប់ដែ

In [ ]:
import pickle as pkl
# save dictionary of intent and entity as pickle file
pkl.dump(dict_intent_entity,open("dict_intent_entity.pkl","wb"))

In [19]:
# drop null values in question column
train_data = dataset[["intent","question"]].dropna()

In [20]:
from sentence_transformers import SentenceTransformer
# load pre-trained model for sentence embedding
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

c:\Users\davan\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\davan\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [21]:
# encode questions
feature_questions = pd.DataFrame(train_data[["question"]].apply(lambda x: model.encode(x[0]),axis=1).values.tolist())

C:\Users\davan\AppData\Local\Temp\ipykernel_14096\2471366086.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  feature_questions = pd.DataFrame(train_data[["question"]].apply(lambda x: model.encode(x[0]),axis=1).values.tolist())


In [22]:
target_intents = train_data[["intent"]]

In [23]:
# change target intents to lowercase
target_intents = target_intents.applymap(lambda x: x.lower())

C:\Users\davan\AppData\Local\Temp\ipykernel_14096\3723570839.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  target_intents = target_intents.applymap(lambda x: x.lower())


In [25]:
from sklearn.preprocessing import LabelEncoder
# encode target intents
le = LabelEncoder()
target_intents = le.fit_transform(target_intents)

c:\Users\davan\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [27]:
# save feature questions and target intents as pickle file
pkl.dump((feature_questions,target_intents),open("dataset.pkl","wb"))

In [28]:
# save label encoder as pickle file
pkl.dump(le,open("label_encoder.pkl","wb"))